# Import Libraries

In [79]:
import requests
from bs4 import BeautifulSoup
import csv
import re

# Web Scrape to get Sun Set and Sun Rise Times

In [80]:
def get_page_info(url, month, year):
    ## Send a GET request to the URL
    response = requests.get(url)

    ## Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parsing HTML into Soup
        soup = BeautifulSoup(response.content, 'html.parser')

        ## Extract information from the page
        table = soup.find(id='as-monthsun').find('tbody')
        rows = table.find_all('tr')

        with open('Data/daylightLA.csv', 'a', newline='') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=',')
            for day, row in enumerate(rows, start=1):
                data = [td.get_text(strip=True) for td in row.find_all('td')[:2]]
                clean_data = [re.sub(r'[: ]', '', text, count=1).split(" ")[0] for text in data]
                if len(clean_data) == 2:
                    clean_data[1] = str(int(clean_data[1]) + 1200)
                    spamwriter.writerow([f'{month:02d}/{day:02d}/{year}', *clean_data])


# Write CSV headers and Format the URL

In [81]:
def populate_csv():
    ## Writting the headers in CSV
    schema = ["Date", "Time AM", "Time PM"]
    with open('Data/daylightLA.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',')
        spamwriter.writerow(schema)

    ## Get info throughout the years
    for year in range(2020, 2024):
        for month in range(1, 13):        
            url = f'https://www.timeanddate.com/sun/usa/los-angeles?month={month}&year={year}'
            get_page_info(url, month, year)

# Execute Script

In [82]:
if __name__ == "__main__":
    populate_csv()